In [2]:
import pandas as pd
import numpy as np
X_train = pd.read_csv("data/x_train.txt",header=None,sep=" ")
y_train = pd.read_csv("data/y_train.txt",header=None,sep=" ")

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import f_classif
import matplotlib.pyplot as plt
import seaborn as sns
f_values, p_values = f_classif(X_train, y_train)


anova_df = pd.DataFrame({
    'Feature': range(len(p_values)),  # numer cechy
    'p_values': p_values
})
anova_df.sort_values('p_values',ascending=False)

c:\Users\Micha\anaconda3\envs\idp\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,Feature,p_values
111,111,9.997676e-01
296,296,9.991506e-01
448,448,9.977965e-01
53,53,9.975327e-01
466,466,9.944021e-01
...,...,...
4,4,7.587568e-117
7,7,1.062914e-119
6,6,1.980223e-134
3,3,2.693812e-135


In [7]:
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
def custom_score(estimator, X, y):
    n_vars = X.shape[1]
    y_pred = estimator.predict(X)
    gain = 10*precision_score(y, y_pred)*len(y_pred)
    return gain - 200*n_vars

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score  # przykładowy custom score
import matplotlib.pyplot as plt

def custom_score(estimator, X, y):
    n_vars = X.shape[1]
    y_pred = estimator.predict(X)
    gain = 10*precision_score(y, y_pred)*len(y_pred)
    return gain - 200*n_vars

custom_scorer = make_scorer(custom_score)

def evaluate_rfe_with_custom_score(X, y, step=1, min_features=5, max_features=20, cv=5):
    results = []

    for n_features in range(min_features, max_features + 1, step):
        print(f"Testowanie {n_features} cech...")

        model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbosity=0)
        selector = RFE(estimator=model, n_features_to_select=n_features, step=0.25, verbose=1)
        selector.fit(X, y)
        X_selected = selector.transform(X)

        scores = cross_val_score(model, X_selected, y, cv=cv, scoring=custom_scorer)
        mean_score = np.mean(scores)
        print(mean_score)
        results.append((n_features, mean_score))

    return results

results = evaluate_rfe_with_custom_score(X_train, y_train)



Testowanie 5 cech...


KeyboardInterrupt: 